In [123]:
exec(open("initKeras.py").read())

In [98]:
# #data preparation
inputDir = "F:\\myProjects\\tim209\\filtered by category\\output\\*.npy"

files = glob.glob( inputDir )
#print( files )

numberOfLabels = len( files ) #each file has a unique label

sources = list()
labelLevelMap = {}
i = 0
for path in files:
    
    label = path.split( "\\" )[-1].replace( ".npy", "" )
    
    sources.append( ( label, path ) )
    
    labelLevelMap[label] = i
    i = i + 1

labels = []
images = []
i = 0
for ( label, path ) in sources:
    
    print( "Processing", label, path )
    
    labelData = np.load( path ).astype( np.float32 ) / 255
    images.append( labelData )
    
    hotVector = np.array( [0] * numberOfLabels, np.float32 ) #must convert to 32 bit. by default it's 64
    hotVector[i] = 1.0
    i = i + 1
    
    hotVectors = [ hotVector ] * labelData.shape[0] #ndarray for single observations, list for all
    labels.append( hotVectors )
    
images = np.vstack(images)
labels = np.vstack( labels )
    
# inputDir = "F:\\myProjects\\tim209\\filtered by category\\output\\"
# labels = []
# images = []
# for k, npy in enumerate([fn for fn in os.listdir(inputDir) if fn.endswith('.npy')]):
#     d = np.load(os.path.join(inputDir, npy))
#     subset = os.path.splitext(npy)[0]
#     images.append(d)
#     hot_vec = [0]*10
#     hot_vec[k] = 1
#     labels += [hot_vec]*d.shape[0]
# images = np.vstack(images)

    

Processing aircraft carrier F:\myProjects\tim209\filtered by category\output\aircraft carrier.npy
Processing airplane F:\myProjects\tim209\filtered by category\output\airplane.npy
Processing alarm clock F:\myProjects\tim209\filtered by category\output\alarm clock.npy
Processing ambulance F:\myProjects\tim209\filtered by category\output\ambulance.npy
Processing angel F:\myProjects\tim209\filtered by category\output\angel.npy
Processing animal migration F:\myProjects\tim209\filtered by category\output\animal migration.npy
Processing ant F:\myProjects\tim209\filtered by category\output\ant.npy
Processing anvil F:\myProjects\tim209\filtered by category\output\anvil.npy
Processing apple F:\myProjects\tim209\filtered by category\output\apple.npy
Processing arm F:\myProjects\tim209\filtered by category\output\arm.npy


In [108]:

print( labels[0] )
print( labels[10000] )
print( labels[30000] )

np.random.seed = 0
seq = np.arange( labels.shape[0] )
np.random.shuffle( seq )
side = int( math.sqrt( images.shape[1] ) )
imagesR = images[ seq ].reshape( ( images.shape[0], side , side, 1  ) )
labelsR = labels[seq]



[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [109]:

#model 1
model = models.Sequential()
model.add( layers.Conv2D( 16, (3,3), activation = activations.relu, input_shape = (28, 28, 1 ) ) )
model.add( layers.MaxPooling2D( (2,2) ) )
model.add( layers.Conv2D( 32, (3,3), activation = activations.relu ))
model.add( layers.MaxPooling2D( (2,2) ) )
model.add( layers.Conv2D( 32, (3,3), activation = activations.relu ))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 32)          9248      
Total params: 14,048
Trainable params: 14,048
Non-trainable params: 0
_________________________________________________________________


In [110]:
model.add( layers.Flatten() )
model.add( layers.Dense( 32, activation = activations.relu ) )
model.add( layers.Dense( 10, activation = activations.softmax ) )


In [119]:
trX, tsX, trY, tsY = train_test_split( imagesR, labelsR, test_size=0.2, random_state=42 )

In [126]:
model.compile( optimizer = optimizers.rmsprop( lr = .001 ), 
             loss = losses.categorical_crossentropy,
             metrics = [ metrics.categorical_accuracy ] )

In [128]:
model.fit( trX, trY, epochs = 3, batch_size = 64 )

Epoch 1/3
80000/80000 [==============================] - 6s 70us/step - loss: 0.4366 - categorical_accuracy: 0.8679
Epoch 2/3
80000/80000 [==============================] - 6s 71us/step - loss: 0.4050 - categorical_accuracy: 0.8782
Epoch 3/3
80000/80000 [==============================] - 6s 70us/step - loss: 0.3830 - categorical_accuracy: 0.8841
